In [58]:
def get_3_clockwise_cup_labels(current_cup_label, cups):
    pos = cups.index(current_cup_label)
    out = []
    for i in range(pos+1, pos+4):
        real_pos = i % len(cups)
        out.append(cups[real_pos])
    return out

# returns label and position (within cups) of the destination cup
def get_destination_cup_with_pos(current_cup_label, cups):
    dest_cup_label = current_cup_label - 1
    cup_set = set(cups)
    max_cup_label = max(cup_set)
    min_cup_label = min(cup_set)
    while dest_cup_label not in cup_set:
        dest_cup_label -= 1
        if dest_cup_label < min_cup_label:  # wrap around to highest value instead
            dest_cup_label = max_cup_label

    dest_cup_pos = cups.index(dest_cup_label)
    return dest_cup_label, dest_cup_pos

def get_next_cup_label(current_cup_label, cups):
    new_pos = (cups.index(current_cup_label)+1) % len(cups)
    return cups[new_pos]

def convert_to_output(cups):
    pos_1 = cups.index(1)
    out = []
    for i in range(pos_1 + 1, pos_1 + 9):
        pos = i % len(cups)
        out.append(cups[pos])
    return ''.join(str(x) for x in out)

def make_move(current_cup_label, cups):
    # grab the 3 cups clockwise and remove from cups
    clockwise_cups = get_3_clockwise_cup_labels(current_cup_label, cups)
    for cup in clockwise_cups:
        cups.remove(cup)
        
    dest_cup_label, dest_cup_pos = get_destination_cup_with_pos(current_cup_label, cups)
    # insert the clockwise cups right after dest_cup_pos
    cups[dest_cup_pos+1:dest_cup_pos+1] = clockwise_cups
    new_current_cup = get_next_cup_label(current_cup_label, cups)
    return new_current_cup, cups

# my input
cups = list(map(int, list('962713854')))
current_cup = cups[0]

for i in range(100):
    current_cup, cups = make_move(current_cup, cups)

print('Final move: %s: %d' % (cups, current_cup))
print('Output: %s' % convert_to_output(cups))

Final move: [5, 4, 3, 2, 9, 7, 8, 1, 6]: 2
Output: 65432978


In [210]:
# okay fine linked lists

class Node:
    def __init__(self, label):
        self.label = label
        self.next = None
        
        
class MyCircularLinkedList:

    # initializes with node labels in order, "current" node as the 1st one
    def __init__(self, labels):
        first_label = labels[0]
        self.current = Node(first_label)
        self.label_to_node_dict = {first_label: self.current}
        
        now_at_node = self.current
        for label in labels[1:]:  # build up the chain
            node = Node(label)
            self.label_to_node_dict[label] = node
            now_at_node.next = node
            now_at_node = node
        
        now_at_node.next = self.current  # link the last one back to the current node
        
        self.in_flight_list = []  # this will keep track of labels that are in flight

    def add_after_label(self, starting_label, labels):
        now_at_node = self.label_to_node_dict[starting_label]
        later_next = now_at_node.next
        for label in labels:
            node = Node(label)
            self.label_to_node_dict[label] = node
            now_at_node.next = node
            now_at_node = node
        now_at_node.next = later_next
    
    def print_from_current(self):
        print(self.current.label, end = ' ')
        node = self.current.next
        while node:
            if node.label == self.current.label:
                break
            print(node.label, end = ' ')
            node = node.next
            
        print('\n')

    def set_current_by_label(self, label):
        self.current = self.label_to_node_dict[label]
        
    def remove_three_after_current(self):
        now_at_node = self.current
        self.in_flight_list = []
        for i in range(3):
            now_at_node = now_at_node.next
            self.in_flight_list.append(now_at_node.label)
            del self.label_to_node_dict[now_at_node.label]

        self.current.next = now_at_node.next
        
    # returns the label of the destination node
    def get_destination_label(self):
        # get the label of current minus one
        dest_label = self.current.label - 1
        while dest_label in self.in_flight_list:
            dest_label -= 1
        if dest_label < 1:
            max_label = len(self.label_to_node_dict.keys()) + len(self.in_flight_list)
            while max_label in self.in_flight_list:
                max_label -= 1
            dest_label = max_label

        dest_node = self.label_to_node_dict[dest_label]
        return dest_node.label
    
    def move(self):
        # remove 3 nodes after current
        self.remove_three_after_current()
        dest_label = self.get_destination_label()
        self.add_after_label(dest_label, self.in_flight_list)
        self.in_flight_list = []
        self.current = self.current.next


In [220]:
cups = list(map(int, list('962713854')))
current_cup = cups[0]

cll = MyCircularLinkedList(cups)
cll.set_current_by_label(current_cup)
print('Starting positions:')
cll.print_from_current()

for i in range(100):
    cll.move()
    
print('Ending positions:')
cll.print_from_current()

Starting positions:
9 6 2 7 1 3 8 5 4 

Ending positions:
2 9 7 8 1 6 5 4 3 



In [252]:
# puzzle 2

# the cups are actually up to 10^6
cups = list(range(1, 1000001))
starting_cups = list(map(int, list('962713854')))
for i in range(len(starting_cups)):
    cups[i] = starting_cups[i]
current_cup = cups[0]

cll = MyCircularLinkedList(cups)
cll.set_current_by_label(9)

In [253]:
for i in range(10000000):
    if i % 1000000 == 0:
        print('at iteration %d' % i)
    cll.move()

at iteration 0
at iteration 1000000
at iteration 2000000
at iteration 3000000
at iteration 4000000
at iteration 5000000
at iteration 6000000
at iteration 7000000
at iteration 8000000
at iteration 9000000


In [254]:
a = cll.label_to_node_dict[1].next.label
b = cll.label_to_node_dict[1].next.next.label

print(a)
print(b)
print(a*b)

778214
369089
287230227046
